In [3]:
import numpy as np

In [17]:
Data = np.loadtxt('../ai_fluidmechanics/data/channel_Re590.txt', skiprows=8)
Data;

In [30]:
#Load of the columns of the data
tke = Data[:,0]
epsilon = Data[:,1]
gradU_flat = Data[:,2:11]
Stress_flat = Data[:,11:]

In [32]:
points = Data.shape[0]
gradU = np.zeros((points, 3,3))
Stresses = np.zeros((points, 3, 3))

In [33]:
for i in range(3):
    for j in range(3):
        gradU[:,i,j] = gradU_flat[:,i*3+j]
        Stresses[:,i,j] = Stress_flat[:,i*3+j]

In [35]:
#Tensors S and R
Sij = np.zeros((points, 3, 3))
Rij = np.zeros((points, 3, 3))

In [37]:
for i in range(points):
    Sij[i,:,:] = 0.5*(gradU[i,:,:]+np.transpose(gradU[i,:,:]))
    Rij[i,:,:] = 0.5*(gradU[i,:,:]-np.transpose(gradU[i,:,:]))

In [38]:
Points2 = Sij.shape[0]
No_invariants = 5
invariants = np.zeros((Points2, No_invariants))

In [39]:
#Create the five invariantes
for i in range(Points2):
    invariants[i,0] = np.trace(np.dot(Sij[i,:,:], Sij[i,:,:]))
    invariants[i,1] = np.trace(np.dot(Rij[i,:,:], Rij[i,:,:]))
    invariants[i,2] = np.trace(np.dot(Sij[i,:,:], np.dot(Sij[i,:,:], Sij[i,:,:])))
    invariants[i,3] = np.trace(np.dot(Rij[i,:,:], np.dot(Rij[i,:,:], Sij[i,:,:])))
    invariants[i,4] = np.trace(np.dot(np.dot(Rij[i,:,:], Rij[i,:,:]), np.dot(Sij[i,:,:], Sij[i,:,:])))    

In [40]:
No_TensorsBasis = 10
T = np.zeros((Points2, No_TensorsBasis, 3, 3))

In [42]:
for i in range(Points2):
    sij = Sij[i,:,:]
    rij = Rij[i,:,:]
    T[i,0,:,:] = sij #Primer tensor T
    T[i,1,:,:] = np.dot(sij, rij) - np.dot(rij, sij)
    T[i,2,:,:] = np.dot(sij, sij) - 1./3*np.eye(3)*np.trace(np.dot(sij, sij))
    T[i,3,:,:] = np.dot(rij, rij) - 1./3*np.eye(3)*np.trace(np.dot(rij, rij))
    T[i,4,:,:] = np.dot(rij, np.dot(sij, sij)) - np.dot(np.dot(sij, sij), rij)
    T[i,5,:,:] = np.dot(rij, np.dot(rij, sij)) + np.dot(sij, np.dot(rij, rij)) - 2./3*np.eye(3)*np.trace(np.dot(sij, np.dot(rij, rij)))
    T[i,6,:,:] = np.dot(np.dot(rij, sij), np.dot(rij,rij)) - np.dot(np.dot(rij,rij), np.dot(sij,rij))
    T[i,7,:,:] = np.dot(np.dot(sij, rij), np.dot(sij, sij)) - np.dot(np.dot(sij, sij), np.dot(rij, sij))
    T[i,8,:,:] = np.dot(np.dot(rij, rij), np.dot(sij, sij)) - np.dot(np.dot(sij, sij), np.dot(rij, rij)) -2./3*np.eye(3)*np.trace(np.dot(np.dot(sij, sij), np.dot(rij, rij)))
    T[i,9,:,:] = np.dot(np.dot(rij, np.dot(sij, sij)), np.dot(rij, rij)) - np.dot(np.dot(rij, np.dot(rij,sij)) , np.dot(sij, sij))
    for j in range(No_TensorsBasis):
        T[i, j, :, :] = T[i, j, :, :] - 1./3*np.eye(3)*np.trace(T[i, j, :, :])    